# Web Scraping www.drogueriascolsubsidio.com

In [1]:
import time
import os
os.chdir(r'C:\Users\Fsalinas\Documents\GitHub\boticarios')
time.sleep(2)
os.chdir('./paquetes')
from connpostgres import conn2
from runSQL import RunDML, RunDDL
time.sleep(2)
os.chdir(r'C:\Users\Fsalinas\Documents\GitHub\boticarios')

In [2]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import urllib.request
from contextlib import closing
from datetime import datetime as dt
import pandas as pd
import json
import random

_________
### Diseño de funciones de carga a PostgreSQL
Generación de funciones de carga de productos a Base de datos (boticarios) de PostgreSQL.

________
***Archivo***: boticarios/postgresql/inicializacion.sql

In [3]:
with open('./postgresql/inicializacion.sql', 'r') as f:
    sql1 = f.read()
print(sql1)

 /*
 * Proyecto: BOTICARIOS.COM
 *
 * Host: localhost
 * Port: 5432
 * Usuarios: (postgres, aadd4455), (fsalinas, fsalinas1547), (cgomez, cgomez4526)
 * Base de datos: boticarios
 * Esquemas: web_scraping
 *
 */
 
CREATE ROLE fsalinas WITH SUPERUSER CREATEDB CREATEROLE LOGIN ENCRYPTED PASSWORD 'fsalinas1547';
CREATE ROLE cgomez WITH SUPERUSER CREATEDB CREATEROLE LOGIN ENCRYPTED PASSWORD 'cgomez4526';
CREATE DATABASE boticarios;
CREATE SCHEMA IF NOT EXISTS web_scraping;
CREATE SCHEMA IF NOT EXISTS datawarehouse;


________
***Archivo***: boticarios/postgresql/fsalinas_webscraping_colsubsidio.sql

In [4]:
with open('./postgresql/fsalinas_webscraping_colsubsidio.sql', 'r') as f:
    sql1 = f.read()
print(sql1)

CREATE TABLE IF NOT EXISTS web_scraping.drog_colsubsidio(
	id						SERIAL,
	url_producto			TEXT PRIMARY KEY,
	fecha_hora_scraping		TIMESTAMP WITHOUT TIME ZONE,
	breadcumb				TEXT,
	titulo					TEXT,
	nombre_imagen			TEXT,
	presentacion			TEXT,
	precio					TEXT,
	descripcion				TEXT,
	atributos				TEXT
);

CREATE TABLE IF NOT EXISTS web_scraping.drog_colsubsidio2(
	id						SERIAL,
	url_producto			TEXT,
	fecha_scraping			TEXT,
	hora_scraping			TEXT,
	titulo					TEXT,
	presentacion			TEXT,
	precio_tachado			NUMERIC,
	precio_final			NUMERIC,
	PRIMARY KEY (url_producto, fecha_scraping)
);


In [5]:
def carga_producto_postgres(dc_prod):
    db = conn2('fsalinas', False)
    sql = f'''
    INSERT INTO web_scraping.drog_colsubsidio (url_producto, fecha_hora_scraping, breadcumb, titulo, nombre_imagen, presentacion, precio, descripcion, atributos)
    SELECT *
    FROM (values{
        tuple([str(x).replace("'","") for x in pd.DataFrame.from_dict(dc_prod, orient='index').T.values.tolist()[0]])
        }) as s(url_producto, fecha_hora_scraping, breadcumb, titulo, nombre_imagen, presentacion, precio, descripcion, atributos)
    '''
    RunDML(sql, db)[0]
    db.close()

In [6]:
def obtiene_data_producto_de_cat(navegador):
    fecha_scraping = dt.now().strftime('%Y-%m-%d')
    hora_scraping = dt.now().strftime('%H:%M:%S')
    soup_productos = BeautifulSoup(navegador.page_source, 'html.parser').find_all('div', {'class':'product-Vitrina-masVendidos js-productVitrineShowcase WishlistModule rendered'})
    
    lista_data_productos = []
    for prod in soup_productos:
        url_producto = prod.find_all('a')[0].get_attribute_list('href')[0]
        titulo = prod.find('p', {'class': 'dataproducto-nameProduct'}).text
        
        try:
            presentacion = prod.find('div', {'class': 'dataproducto-info dataproducto-Presentacion'}).text.strip()
        except:
            presentacion = 'N/A'
        
        try:
            precio_tachado = float(prod.find('div', {'class': 'precioTachadoVitrina'}).text.replace('Antes: $','').replace('.','').replace(',','.'))
        except:
            precio_tachado = 0
        
        try:
            precio_final = float(prod.find('p', {'class': 'dataproducto-bestPrice'}).text.replace('$','').replace('.','').replace(',','.'))
        except:
            precio_final = 0
        
        lista_data_productos += [{
            'url_producto': url_producto,
            'fecha_scraping': fecha_scraping,
            'hora_scraping': hora_scraping,
            'titulo': titulo,
            'presentacion': presentacion,
            'precio_tachado': precio_tachado,
            'precio_final': precio_final
        }]
    
    return pd.concat([pd.DataFrame.from_dict(x, orient='index').T for x in lista_data_productos], axis=0, ignore_index=True)

In [7]:
def carga_producto_postgres2(df_prods):
    db = conn2('fsalinas', False)
    sql = f'''
    INSERT INTO web_scraping.drog_colsubsidio2 (url_producto, fecha_scraping, hora_scraping, titulo, presentacion, precio_tachado, precio_final)
    SELECT *
    FROM (values{str([tuple(x) for x in df_prods.values.tolist()]).replace("[","").replace("]","")}) as s(url_producto, fecha_scraping, hora_scraping, titulo, presentacion, precio_tachado, precio_final)
    '''
    resultado = str(RunDML(sql, db))
    db.close()
    return resultado

_________
### Web Crawler Categorias
Extraé urls de categorias que contienen las url de los productos para posteriormente extraer información de producto.

In [8]:
# Define rutas a urls y archivos
chromedriver = './web_scraping/chromedriver/chromedriver.exe'
url_principal = 'https://www.drogueriascolsubsidio.com'

# Define si el navegador estará visible durante el proceso
hide_browser = False

# Aplica opciones al navegador para evitar cargar recursos innecesarios
options = Options()
options.add_argument('--ignore-certificate-errors')
if hide_browser: options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('prefs',{'profile.managed_default_content_setings.images':2})

# Crea el objeto del navegador con el que se realizará la interacción
with closing(Chrome(executable_path = chromedriver, options=options)) as navegador:

    # Navega a la URL principal donde se extraerán las URLs de categorias y subcategorias de producto
    navegador.get(url_principal)
    soup = BeautifulSoup(navegador.page_source, 'html.parser')

    # Obtiene una lista de las etiquetas "ul" asociadas a las categorias de producto
    # Fuente: investigación en las particularidades de la construcción del sitio web, es suceptible a fallos en caso de que
    # la estructura (etiquetas) del sitio web cambie.
    lista_categorias = soup.find_all('ul',{'class':'categoria-container'})
    dc_cat_url = {cat.find_all('a')[1].text.lower().replace('ver ', ''):cat for x, cat in enumerate(lista_categorias)}

    #Extrae la lista de URLs de las etiquetas "ul"
    lista_urls = []
    for cat in dc_cat_url.keys():
        lista_urls += [x.get_attribute_list('href')[0] for x in dc_cat_url[cat].find_all('a')]

    lista_urls = list(dict.fromkeys(lista_urls))

# Guarda la lista de URLs en un archivo csv
with open('./web_scraping/data/cat_urls.csv', 'w+') as f:
    f.write('\n'.join(lista_urls))

_________
### Web Crawler Productos
Extraé urls de productos a partir de las URLs de categorias del archvio csv.

In [9]:
# Define rutas a urls y archivos
chromedriver = './web_scraping/chromedriver/chromedriver.exe'
url_principal = 'https://www.drogueriascolsubsidio.com'

# Define si el navegador estará visible durante el proceso
hide_browser = False

# Aplica opciones al navegador para evitar cargar recursos innecesarios
options = Options()
options.add_argument('--ignore-certificate-errors')
if hide_browser: options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('prefs',{'profile.managed_default_content_setings.images':2})

In [10]:
# Leer el archivo con las url de categorias
with open('./web_scraping/data/cat_urls.csv', 'r') as f:
    cat_urls = f.read()
cat_urls = cat_urls.split('\n')

In [11]:
# Función para obtener todas las url de producto ubicadas en una url
def obtiene_url_productos(navegador):
    soup = BeautifulSoup(navegador.page_source, 'html.parser')
    urls_prod = [x.get_attribute_list('href')[0] for x in soup.find_all('a') if x.get_attribute_list('href')[0]!=None and x.get_attribute_list('href')[0][-2:].lower()=='/p']
    urls_prod = [x.replace(url_principal, '') for x in urls_prod]
    return list(dict.fromkeys(urls_prod))

In [12]:
def obtiene_urls_producto(navegador, url, tiempo_espera_scroll=2):
    # Navega a cada URL de categoria o subcategoria donde se extraerán las URLs de producto
    navegador.get(url)

    # Obtener el tamaño de la pagina cargada
    ultima_altura = navegador.execute_script("return document.body.scrollHeight")

    lista_urls_producto, scroll_nro = [], 1
    while True:
        print(f'{dt.now().strftime("%H:%M:%S")} Scroll nro: {scroll_nro:,.0f}', end = '\r')
        # Scroll hasta el final de la pagina
        navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # captura las urls de la pagina hasta donde está cargada
        lista_urls_producto += obtiene_url_productos(navegador)

        # esperar a que la pagina cargue
        time.sleep(tiempo_espera_scroll)

        # Calcula la nueva altura de la pagina despues de hacer scroll y lo compara con la latura anterior
        nueva_altura = navegador.execute_script("return document.body.scrollHeight")
        if nueva_altura == ultima_altura:
            break
        ultima_altura = nueva_altura
        scroll_nro += 1
    
    df_prods = obtiene_data_producto_de_cat(navegador)
    # print('Resultado de carga:\n' + '\n'.join([carga_producto_postgres2(df_prods.iloc[x:x+1, :])[:20] for x in df_prods.index]))

    lista_urls_producto = list(dict.fromkeys(lista_urls_producto))
    return lista_urls_producto

In [13]:
# Crea el objeto del navegador con el que se realizará la interacción
# Duración aproximada: 30 minutos
with closing(Chrome(executable_path = chromedriver, options=options)) as navegador:
    lista_urls_productos = []
    for cat in cat_urls:
        print(f'{dt.now().strftime("%H:%M:%S")} Procesando: {cat}')
        lp = obtiene_urls_producto(navegador, url_principal + cat, 2)
        print(f'{dt.now().strftime("%H:%M:%S")} Numero de urls de producto conseguidas:{len(lp):,.0f}')
        lista_urls_productos += lp

lista_urls_productos = list(dict.fromkeys(lista_urls_productos))
print(f'{dt.now().strftime("%H:%M:%S")} Numero de urls de producto totales conseguidas:{len(lista_urls_productos):,.0f}')
print('Primeras 10 urls de la lista completa:')
lista_urls_productos[:10]

14:18:05 Procesando: /medicamentos
Resultado de carga:: 2
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
14:18:16 Numero de urls de producto conseguidas:33
14:18:16 Procesando: /medicamentos/venta-libre
Resultado de carga:: 33
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
('KO: duplicate key 
(

['/advil-max-400-mg-capsula-blanda-7702132008611/p',
 '/systane-complete-gotas-lubricantes-300651481228/p',
 '/benzirin-verde-aseptic-solucion-bucal-7702057012724/p',
 '/7702870002964gastrum-10-mg-tabletamedicamentosbienestar-digestivolafrancolfamotidina-7702870002964/p',
 '/proteina-barbarus-healthy-sports-19962798764/p',
 '/engystol-tableta-7707336720598/p',
 '/optive-fusion-solucion-oftalmica-7707236676087/p',
 '/7702132004460pfizerbienestar-digestivomedicamentosmareol-50-mg-tabletadimenhidrinato-7702132004460/p',
 '/7702132004644oferta-chapstick-medicado-duo-pack-barra-precio-especialmedicamentoscuidado-facialna-7702132004644/p',
 '/dolex-forte-nf-500-mg65-mg-tableta-recubierta-7703363005462/p']

In [ ]:
print(f'{dt.now().strftime("%H:%M:%S")} Numero de urls de producto totales conseguidas:{len(lista_urls_productos):,.0f}')
print('Primeras 10 urls de la lista completa:')
lista_urls_productos[:10]

In [ ]:
# Guarda la lista de URLs en un archivo csv
with open('./web_scraping/data/prod_urls.csv', 'w+') as f:
    f.write('\n'.join(lista_urls_productos))

_________
### Web Scraping Productos
Extraé data de cada url de producto a partir de las URLs de producto del archvio csv.

In [ ]:
# Define rutas a urls y archivos
chromedriver = './web_scraping/chromedriver/chromedriver.exe'
url_principal = 'https://www.drogueriascolsubsidio.com'

# Define si el navegador estará visible durante el proceso
hide_browser = False

# Aplica opciones al navegador para evitar cargar recursos innecesarios
options = Options()
options.add_argument('--ignore-certificate-errors')
if hide_browser: options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('prefs',{'profile.managed_default_content_setings.images':2})

In [ ]:
# Leer el archivo con las url de categorias
with open('./web_scraping/data/prod_urls.csv', 'r') as f:
    prod_urls = f.read()
prod_urls = prod_urls.split('\n')

In [ ]:
def descarga_imagen(navegador, nombre):
    url_imagen_producto = BeautifulSoup(navegador.page_source, 'html.parser').find_all('img', {'class': 'class-img-product'})[0].get_attribute_list('src')[0]
    nombre_imagen = f'./web_scraping/data/img/{nombre}.jpg'
    urllib.request.urlretrieve(url_imagen_producto, nombre_imagen)
    return nombre_imagen

In [ ]:
def obtiene_data_producto(navegador):
    soup_producto = BeautifulSoup(navegador.page_source, 'html.parser').find_all('div', {'class': 'container-data'})[0]
    
    fecha_hora_scraping = dt.now().strftime('%Y-%m-%d %H:%M:%S')
    
    try:
        titulo = soup_producto.find_all('h2')[0].text
        precio = {str(x).split('class="')[1].split('"')[0]:x.text for x in soup_producto.find_all('div', {'class': 'precio'})[0].find_all('div')}
    except:
        return {
            'url_producto': url_producto,
            'fecha_hora_scraping': fecha_hora_scraping,
            'breadcumb': 'no encontrado',
            'titulo': 'no encontrado',
            'nombre_imagen': 'no encontrado',
            'presentacion': 'no encontrado',
            'precio': 'no encontrado',
            'descripcion': 'no encontrado',
            'atributos': 'no encontrado'
        }
    
    try:
        breadcumb = '|'.join([x.text.lower() for x in BeautifulSoup(navegador.page_source, 'html.parser').find_all('div', {'class': 'bread-crumb'})[1].find_all('li')])
        presentacion = soup_producto.find_all('div', {'class': 'ContentPresentaticones'})[0].text
        descripcion = soup_producto.find_all('div', {'class': 'productDescription'})[0].text
        atributos = [tuple(x.text.split(':')) for x in soup_producto.find_all('div', {'class': 'divProduct-especificaciones'})[0].find_all('div', {'class': 'especificaciones_item'})]
    except:
        return {
            'url_producto': 'no encontrado',
            'fecha_hora_scraping': fecha_hora_scraping,
            'breadcumb': 'no encontrado',
            'titulo': titulo,
            'nombre_imagen': titulo + '.jpg',
            'presentacion': 'no encontrado',
            'precio': precio,
            'descripcion': 'no encontrado',
            'atributos': 'no encontrado'
        }
    
    try:
        nombre_imagen = descarga_imagen(navegador, ''.join([x for x in titulo.replace(' ', '_') if x.isalnum() or x=='_']))
    except:
        return {
            'url_producto': url_producto,
            'fecha_hora_scraping': fecha_hora_scraping,
            'breadcumb': breadcumb,
            'titulo': titulo,
            'nombre_imagen': titulo + '.jpg',
            'presentacion': presentacion,
            'precio': precio,
            'descripcion': descripcion,
            'atributos': atributos
        }
    
    return {
        'url_producto': url_producto,
        'fecha_hora_scraping': fecha_hora_scraping,
        'breadcumb': breadcumb,
        'titulo': titulo,
        'nombre_imagen': nombre_imagen,
        'presentacion': presentacion,
        'precio': precio,
        'descripcion': descripcion,
        'atributos': atributos
    }

In [ ]:
# Crea el objeto del navegador con el que se realizará la interacción
# navegador = Chrome(executable_path = chromedriver, options=options)
random.shuffle(prod_urls)
with closing(Chrome(executable_path = chromedriver, options=options)) as navegador:
    for prod_url in prod_urls:
        fecha = dt.now().strftime('%Y%m%d')
        if not os.path.exists(f'./web_scraping/data/json/{fecha}'): os.makedirs(f'./web_scraping/data/json/{fecha}')
        json_path = f'./web_scraping/data/json/{fecha}'
        
        print(f'{dt.now().strftime("%H:%M:%S")} Procesando: {prod_url}')
        url_producto = url_principal + prod_url
        navegador.get(url_producto)
        time.sleep(2)
        dc_prod = obtiene_data_producto(navegador)
        carga_producto_postgres(dc_prod)

        json_data = json.dumps(dc_prod, indent=4, sort_keys=False)
        with open(f'{json_path}/{dc_prod["nombre_imagen"].split("/")[-1].replace(".jpg",".json")}', mode='w+', encoding='latin-1') as f:
            f.write(json_data)